In [ ]:
# Dependencies.
import pandas as pd

In [ ]:
# Load clue data file.
clueDf = pd.read_csv("clueAnswers.csv",encoding="utf8",index_col=0)

In [ ]:
# During manual analysis of clues, some were found that referenced images with missing links (thus not being picked up during parsing).
# Some categories were also found to be to difficult to reassign. Both were assigned category "REMOVE".
# We can remove the "REMOVE" columns from the data files used for the web application, until a proper fix can be found.
clueDf = clueDf.loc[clueDf["categoryAssigned"]!="REMOVE"]

In [ ]:
# Building the normalized category data file.
categoryIDList = []
roundList = []
categoryList = []
categoryCommentList = []
categoryAssignedList = []
subcategoryAssignedList = []
questionSetList = []
for i in clueDf['categoryID'].unique():
    categoryIDList.append(i)
    roundList.append(clueDf.loc[clueDf['categoryID'] == i]['round'].tolist()[0])
    categoryList.append(clueDf.loc[clueDf['categoryID'] == i]['category'].tolist()[0])
    categoryCommentList.append(clueDf.loc[clueDf['categoryID'] == i]['categoryComment'].tolist()[0])
    categoryAssignedList.append(clueDf.loc[clueDf['categoryID'] == i]['categoryAssigned'].tolist()[0])
    subcategoryAssignedList.append(clueDf.loc[clueDf['categoryID'] == i]['subcategoryAssigned'].tolist()[0])
    match = clueDf.loc[clueDf['categoryID'] == i]['match'].tolist()[0]
    if match > 0.6:
        questionSetList.append(1)
    elif match > 0.4:
        questionSetList.append(2)
    else:
        questionSetList.append(3)
categoryDf = pd.DataFrame()
categoryDf["categoryID"] = categoryIDList
categoryDf["round"] = roundList
categoryDf["category"] = categoryList
categoryDf["categoryComment"] = categoryCommentList
categoryDf["categoryAssigned"] = categoryAssignedList
categoryDf["subcategoryAssigned"] = subcategoryAssignedList
categoryDf["questionSet"] = questionSetList
categoryDf.set_index("categoryID",inplace=True)
# Build the restricted question set dataframes.
categoryDf2 = categoryDf[categoryDf["questionSet"]<3].copy()
categoryDf1 = categoryDf[categoryDf["questionSet"]<2].copy()
# Remove the questionSet column to save space.
categoryDf.drop(columns=["questionSet"],inplace=True))
categoryDf2.drop(columns=["questionSet"],inplace=True))
categoryDf1.drop(columns=["questionSet"],inplace=True))
# Adjust the paths as you see fit.
categoryDf.to_csv("category3.csv")
categoryDf2.to_csv("category2.csv")
categoryDf1.to_csv("category1.csv")

In [ ]:
# Load metadata file. Now that we are finished with manual checking we no longer need a filename column.
metadataDf = pd.read_csv("metadata.csv",encoding="utf8",index_col="showID")
showIDList = []
for i in clueDf["filename"].tolist():
    showIDList.append(metadataDf[metadataDf["filename"]==i]["showID"].values[0])
clueDf['showID'] = showIDList
metadataDf.drop(columns=["filename"],inplace=True))
metadataDf.to_csv("metadata.csv")

In [ ]:
# Keep only the columns unique to a clue.
clueDf = clueDf[["clueID","categoryID","showID","value","clue","answer1","answer2","answer3","answer4","answer5","answer6","answer7","answer8","answer9","answer10","answer11"]]
# This could have been done in parsing.ipynb as well.
clueDf.set_index("clueID",inplace=True)
# Build restricted question set dataframes for clues.
clueDf2 = clueDf[clueDf["categoryID"].isin(categoryDf2.index.tolist())]
clueDf1 = clueDf[clueDf["categoryID"].isin(categoryDf1.index.tolist())]
# Save files.
clueDf.to_csv("clue3.csv")
clueDf2.to_csv("clue2.csv")
clueDf1.to_csv("clue1.csv")